In [153]:
import os 
!pwd

/home/mehrajofficial000/venv/ml_project


In [154]:
#os.chdir("../")

In [155]:
!pwd

/home/mehrajofficial000/venv/ml_project


In [156]:
#entity 
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig: 
    root_dir: Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data : Path
    params_epochs : int
    params_batch_size : int
    params_is_augmentation : bool
    params_image_size : list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_path: Path

In [157]:
#Configurations

from Classifier.constants import *
from Classifier.utils.utilities import read_yaml, create_directories
import tensorflow as tf
from pathlib import Path

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.training.root_dir])


    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_path)
        create_directories([
            Path(model_checkpoint_dir),
            Path(config.tensorboard_root_log_dir)
            ])
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_path=Path(config.checkpoint_model_path)
        )
        return prepare_callbacks_config


    def get_training_config(self) -> TrainingConfig:
        config = self.config.training
        prepare_base_model_config = self.config.prepared_base_model

        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([Path(config.root_dir)])
        


        training_config = TrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            updated_base_model_path=Path(prepare_base_model_config.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=self.params.EPOCHS,
            params_batch_size=self.params.BATCH_SIZE,
            params_is_augmentation=self.params.AUGMENTATION,
            params_image_size=self.params.IMAGE_SIZE
            params_learning_rate = self.params.LEARNING_RATE
        )
        return training_config
        

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1686962850.py, line 49)

In [ ]:
#components
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 
import time 
import logging

class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at-{timestamp}"
        )
        logging.info(f"Tensorboard log dir: {tb_running_log_dir}")
        return tf.keras.callbacks.TensorBoard(
            log_dir=tb_running_log_dir,
            histogram_freq=1, #Problem might occur 
            write_graph=True,  
            write_images=True,
            update_freq="epoch",
        )
    
    @property
    def _create_checkpoint_callback(self):
        logging.info(f"Checkpoint dir: --->>>>> creating checkpoint dir ----->")
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_path,
            monitor = "val_accuracy", #problems might occur 
            save_best_only = True,
        )
    
    def get_callbacks(self):
        logging.info("Preparing callbacks")
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callback
        ]
    

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf 
import time 
import logging

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        
    def get_base_model(self):
    self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    # ✅ Recompile the model with a new optimizer instance
    self.model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.SGD(),
        metrics=['accuracy']
    )

    logging.info(f"Base model loaded and recompiled from {self.config.updated_base_model_path}")

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2,  # 20% for validation
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear",
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            class_mode = "sparse",
            shuffle = False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True,
                fill_mode = "nearest",
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
        

        self.train_generator = train_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "training",
            class_mode = "sparse",
            shuffle = True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path :Path, model: tf.keras.Model):
        logging.info(f"Saving model at {path}")
        model.save(path)
    
    def train(self, callback_List :list):
        logging.info("Training started")
        self.steps_per_epoch = self.train_generator.samples // self.config.params_batch_size
        self.validation_steps = self.valid_generator.samples // self.config.params_batch_size
        
        self.model.fit(
            self.train_generator,
            steps_per_epoch = self.steps_per_epoch,
            validation_data = self.valid_generator,
            validation_steps = self.validation_steps,
            epochs = self.config.params_epochs,
            callbacks = callback_List
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

IndentationError: expected an indented block after function definition on line 5 (2037242585.py, line 6)

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [ ]:
#pipeline

try :
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callbacks = prepare_callbacks.get_callbacks()
    logging.info(f"Callbacks: {callbacks}")


    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_List=callbacks
    )
except Exception as e:
    logging.error(f"Error in loading configuration_training: {e}")
    raise e

[2025-08-02 12:51:39,352] INFO: utilities: YAML file config/config.yaml loaded successfully.]
[2025-08-02 12:51:39,356] INFO: utilities: YAML file params.yaml loaded successfully.]
[2025-08-02 12:51:39,358] INFO: utilities: Directory craeted at artifacts/training]
[2025-08-02 12:51:39,360] INFO: utilities: Directory craeted at artifacts/prepare_callbacks/checkpoint_dir]
[2025-08-02 12:51:39,361] INFO: utilities: Directory craeted at artifacts/prepare_callbacks/tensorboard_log_dir]
[2025-08-02 12:51:39,362] INFO: 3489171288: Preparing callbacks]
[2025-08-02 12:51:39,365] INFO: 3489171288: Tensorboard log dir: artifacts/prepare_callbacks/tensorboard_log_dir/tb_logs_at-2025-08-02-12-51-39]
[2025-08-02 12:51:39,368] INFO: 3489171288: Checkpoint dir: --->>>>> creating checkpoint dir ----->]
[2025-08-02 12:51:39,370] INFO: 842896744: Callbacks: [<keras.src.callbacks.tensorboard.TensorBoard object at 0x7f27dc146b90>, <keras.src.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f282824a9

[2025-08-02 12:51:41,465] WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2025-08-02 12:51:41,523] ERROR: 842896744: Error in loading configuration_training: 'TrainingConfig' object has no attribute 'params_learning_rate']


AttributeError: 'TrainingConfig' object has no attribute 'params_learning_rate'